In [1]:
#imports for machine learning and parsing
import pandas as pd
import seaborn as sns
from sklearn.model_selection import train_test_split # To split our data for fair testing
from sklearn.linear_model import LogisticRegression, LinearRegression  # The model itself!
from sklearn import preprocessing #LabelEncoders
from sklearn.preprocessing import PolynomialFeatures
from sklearn.base import RegressorMixin
from sklearn.ensemble import RandomForestClassifier
import math #for ceil
import json

import pickle
from sklearn import datasets
#server imports
from flask import Flask
from flask import request
from flask import jsonify
from flask_cors import CORS

#for typechecking
from typing import Sequence, Union
import numpy as np

In [2]:
#loading our dataframe
diabetes_dataframe = pd.read_csv("./datasets/diabetes_prediction_dataset.csv")



In [3]:
#replacements
#To convert categorical data into numerical data, I googled it and you use labelencoding.
genderEncoder = preprocessing.LabelEncoder()
smokingEncoder = preprocessing.LabelEncoder()

diabetes_dataframe["gender"] = genderEncoder.fit_transform(diabetes_dataframe["gender"])
diabetes_dataframe["smoking_history"] = smokingEncoder.fit_transform(diabetes_dataframe["smoking_history"])


1

In [4]:

#splitting
features = diabetes_dataframe[['gender', 'age', 'hypertension', 'heart_disease', 'smoking_history', 'bmi', 'HbA1c_level', 'blood_glucose_level']]
outputs = diabetes_dataframe[['diabetes']].values.ravel()

print(outputs)

features_train, features_test, outputs_train, outputs_test = train_test_split(features, outputs, test_size=0.1, random_state=1845)

[0 0 0 ... 0 0 0]


In [5]:
class Model:
    def __init__(self, model: RegressorMixin, transformModel=True):
        self.model = model
        if transformModel: model.fit(features_train, outputs_train)
    
    def test(self, model: None, features=features_test, outputs=outputs_test) -> float:
        if model is None: model = self.model
        return model.score(features, outputs)
    
    def predict(self, features: Union[Sequence[Sequence[float]], np.ndarray]):
        return self.model.predict(features)

class LinearModel(Model):
    def __init__(self, transformModel=True):
        super().__init__(LinearRegression(), transformModel)

class LogisticModel(Model):
    def __init__(self, transformModel=True):
        super().__init__(LogisticRegression(max_iter=1000), transformModel)

class RandomForestModel(Model):
    def __init__(self, trees: int,transformModel=True):
        super().__init__(RandomForestClassifier(n_estimators=trees), transformModel)

class PolynomialModel(Model):
    def __init__(self, maxDegree: int):
        maxModel = None
        score = -1
        self.poly: PolynomialFeatures = None
        for degree in range(1,maxDegree+1):
            iterFeatures = PolynomialFeatures(degree=degree, include_bias=False)
            polyTrainedFeatures = iterFeatures.fit_transform(features_train)

            iterLinear  = LinearModel(transformModel=False)
            iterLinear.model.fit(polyTrainedFeatures, outputs_train)
    

            iterScore = PolynomialModel.test(None,iterFeatures, iterLinear.model)
            if iterScore > score:
                maxModel = iterLinear
                score = iterScore
                self.poly = iterFeatures
                
        super().__init__(maxModel.model, False)

    def test(self, polyFeatures: PolynomialFeatures=None, model: LinearRegression=None) -> float:
        if model is None: model = self.model
        if polyFeatures is None: polyFeatures = self.poly
        
        return model.score(polyFeatures.transform(features_test), outputs_test)
    
    def predict(self, features: Union[Sequence[Sequence[float]], np.array]):
        return self.model.predict(self.poly.transform(features))


In [6]:
#model preperation and training
models = {
    "linear": LinearModel(),
    "logistic": LogisticModel(1000),
    "randomForest" : RandomForestModel(500),
    "polynomial" : PolynomialModel(5)
}

KeyboardInterrupt: 

In [ ]:
class Predictor:

    dataframe: pd.DataFrame

    def __init__(self, dataframe: pd.DataFrame, model: Model=None):
        self.dataframe = dataframe
        self.model = model

    def predict(self):
        return self.model.predict(self.dataframe)
    
    def predict_proba(self):
        if(hasattr(self.model.model, "predict_proba")):
            return [self.model.model.predict_proba(self.dataframe)[0][1]]
        else:
            return self.predict()
    
    @staticmethod
    def parse_json(unparsed_json: str):
        parsed_json = json.loads(unparsed_json)
        return Predictor.json_to_frame(parsed_json)
    
    @staticmethod
    def json_to_frame(parsed_json: dict):
        return Predictor(pd.DataFrame([parsed_json]))

    def convert_binary_values(self):
        self.dataframe["gender"] = genderEncoder.transform(self.dataframe["gender"])
        self.dataframe["smoking_history"] = smokingEncoder.transform(self.dataframe["smoking_history"])

        return self

    def add_model(self, model: Model):
        self.model = model
        return self
    


# jsonPredict = Predictor.parse_json("""{
#     \"gender\" : \"Male\",
#     \"age\" : 12.0,
#     \"hypertension\": 1,
#     \"heart_disease\": 1,
#     \"smoking_history\": \"current\",
#     \"bmi\" : 23.86,
#     \"HbA1c_level\": 4.8,
#     \"blood_glucose_level\": 157.0
# }""").convert_binary_values().predict().tolist()

# print(jsonPredict)



In [ ]:
#Serverside Setup

app = Flask(__name__)
CORS(app)




In [ ]:
#App Route
@app.route("/generate", methods=["POST"])
def generate():
    clientJson = request.json

    model = clientJson["model"]
    features = clientJson["features"]

    if(models.get(model, None) is None):
        return jsonify({"result": f"The {model} model doesn't exist."})

    currModel = Predictor.json_to_frame(features).convert_binary_values().add_model(models[model])

    prediction = float(currModel.predict()[0])
    probability = float(currModel.predict_proba()[0])

    

    return jsonify({"result": [prediction, probability]})

In [ ]:
#Run

if __name__ == '__main__':
    app.run()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit
127.0.0.1 - - [25/Jun/2025 23:13:49] "OPTIONS /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:13:49] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:13:57] "OPTIONS /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:13:57] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:15:00] "OPTIONS /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:15:00] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:15:03] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:15:14] "OPTIONS /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:15:14] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:17:22] "OPTIONS /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:17:22] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:17:40] "OPTIONS /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 23:17:41] "POST /generate HTTP/1.1" 200 -
127.0.0.1 - - [25/Jun/2025 2